## MATH6380 Project 2: DCF-Net Exploration

**Kejing Yin** and **Dong Qian**  
*Department of Computer Science, Hong Kong Baptist University*

**Note**: To run the notebook, Python >= 3.6, PyTorch >= 0.4.1 and torchvision are required.

## 1  Introduction

Convolutional Neural Networks (CNNs) have been one of the most successful models in machine learning and artificial intelligence. Remarkable progress in computer vision has been achieved by the successful structure design of CNNs, such as AlexNet [1], VGGNet [2], ResNet [3] and DenseNet [4]. The filters in CNNs corresponds to the weights that are learned from a large number of data. These filters are typically randomly initialized and then updated by using back-propagation training algorithm. As a result, the learned filters have no specific structures and could not be interpreted by humans. 

The Scattering Network (ScatNet) [5] was designed to use pre-fixed weights in CNNs for interpretability and it showed capacity to learn translation-invariant representations. Filters with improved properties have an impact on the efficiency and interpretability of CNNs, and help to understand deep neural networks.

To leverage filter generation mechanism of CNNs and ScatNet, the Decomposed Convolutional Filters network (DCF-Net) [6] decomposes convolutional filters in CNNs as a pre-fixed Fourier-Bessel (FB) bases, where the expansion coefficients are learned from the data in a supervised learning manner. In this way, it can lead to significant redundancy in the learned model. Experimental results showed the DCFNet yielded comparable and even improved classification accuracy on image datasets.

In this project, we experimentally compare DCFNet with ResNet18 and VGGNet16 (with batch nomalization, VGGNet16_bn) for object classification task. We observe that on various object recognition datasets the classification accuracy are maintained with a significant reduction of the number of parameters, and the performance of Fourier-Bessel (FB) bases is constantly superior.


### Contribution
Both authors contribute equally to this project. More concretly:
- **Kejing Yin** compared DCFNet with ResNet18.  
- **Dong Qian** compared DCFNet with VGGNet16_bn.  

The discussion on the results and the writing of the report are done by both authors.

### Structure of the Report
In section 2, we introduce the dataset we used in this report. In section 3, we briefly introduce the DCF-Nets, VGG16 and ResNet18 models explored in this report followed by the object classification using the discussed models in section 4. Finally in section 5, we conclude the report and discuss the obtained results.

In [1]:
import os
from pathlib import Path

import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import models, transforms

from vgg_dcf import vgg16_bn as dcf_vgg16bn
from resnet_dcf import resnet18 as dcf_resnet18
from torchvision.models import vgg16_bn as cnn_vgg16bn
from torchvision.models import resnet18 as cnn_resnet18

In [2]:
# Prepare the computational envrionment
device = torch.device('cuda')

data_dir = Path('./data')
data_dir.mkdir(exist_ok=True)

results_dir = Path('./results')

## 2 Datasets

we use CIFAR-10 dataset in this project. The CIFAR-10 dataset consists of 60,000 images divided into 10 classes, with 6000 images per class. The classes include airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck. There are 50,000 training samples and 10,000 test samples. The images in CIFAR-10 are of size $3 \times 32 \times 32$, i.e. 3-channel color images of $32 \times 32$ pixels in size. The CIFAR-10 dataset is available at https://www.cs.toronto.edu/~kriz/cifar.html.

Here are the images in the training dataset:

<img src="./figs/CIFAR.png" width="400">

In [4]:
# download data if the data do not exist.
CIFAR10(data_dir, download=True)
print('Data are well prepared.')

Files already downloaded and verified
Data are well prepared.


## 3  DCFNet Model
We first briefly introduce the DCF-Net, VGG16 and ResNet18 models that are used in this project.

### 3.1  DCF-Net

Suppose that the output at the $l$-th layer of CNNs can be written as $\{x^{(l)}(u, \lambda)\}_{u \in \mathbb{R}^{2}, \lambda \in [M_{l}]}$, where $M_{l}$ is the number of channels in that layer. The CNNs with $L$ layers are written as a mapping from $\{x^{(0)}(u, \lambda)\}_{u \in \mathbb{R}^{2}, \lambda \in [M_{0}]}$ to $\{x^{(L)}(u, \lambda)\}_{u \in \mathbb{R}^{2}, \lambda \in [M_{L}]}$ through a nonlinear mapping $x^{(l)}(u, \lambda) = \sigma ( x_{\frac{1}{2}}^{(l)}(u, \lambda) + b^{(l)}(\lambda))$, where $\sigma$ is a nonlinear activation function, e.g., ReLU, and 

$$x_{\frac{1}{2}}^{(l)}(u, \lambda) = \sum_{\lambda^{'} = 1}^{M_{l-1}} \int W_{\lambda^{'}, \lambda}^{(l)} (v^{'}) x^{(l-1)} (u + v^{'}, \lambda^{'}) dv^{'}$$
where the filters $W_{\lambda^{'}, \lambda}^{(l)} (u)$ and the biases $b^{(l)}$ are the parameters of CNNs.

The CNNs represent filters as vectors of the size of the local patches, e.g., $3 \times 3$ or $5 \times 5$ image patches, which is equivalent to expanding filters under the delta bases. The DCF layers represent the convolutional filters as a truncated expansion under basis functions which are not learned through the training process. Specifically, suppose that the convolutional filters $W_{\lambda^{'}, \lambda}(u)$ at certain layer, after a proper rescaling of the spatial variable, are supported on the unit disk $D$ in $\mathbb{R}^{2}$. Given a bases ${\psi_{k}}_{k}$ of the space $L^{2}$, the filters can be represented as

$$W_{\lambda^{'}, \lambda}(u) = \sum_{k=1}^{K}(a_{\lambda^{'}, \lambda})_{k} \psi_{k}(u)$$
where $K$ is the trunction. The decomposition can be viewed as a two-step scheme of a convolutional layer:

1. $\Psi$-step: The input is convolved with each of the pre-fixed basis $\psi_{k}, k = 1, \ldots, K$. The convolution for each input channel is independent from other channels and thus it is computationally efficient.

2. $a$-step: The intermediate output is linearly transformed by an effectively fully-connected weight matrix $(a_{\lambda^{'}, \lambda})_{k}$, which can be learned from the data.

Motivated by classical results of harmonic analysis, FB bases are used in DCF-Net, with which the regularity of the filters $W_{\lambda^{'}, \lambda}$ can be imposed though constraining the magnitude the coefficients $\{(a_{\lambda^{'}, \lambda})_{k}\}_{k}$. Some Fourier-Bessel bases are illustrated as follows.

<img src="./figs/FB.png" width="500">


### 3.2 VGGNet-16

The VGGNet [2] improves the AlexNet [1] by replacing large kernel-sized filters ($11 \times 11$ and $5 \times 5$ in the first and second convolutional layer) with multiple $3 \times 3$ kernel-sized filters one after another. With a given receptive field, multiple stacked smaller-sized kernels are better than those with a larger-sized kernel, because multiple non-linear layers increase the depth of the network which enables it to learn more complex and abstract representations, but at a lower cost.

<img src="./figs/VGGNet.png" width="600">

In this project, we use the VGGNet-16 with batch normalization (VGGNet-16_bn). The implementation of CNN-VGG16 model is from [torchvision](https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py) and that of the DCF-VGG16 model is adapted based on [DCFNet-Pytorch](https://github.com/ZeWang95/DCFNet-Pytorch).

### 3.3 ResNet-18

The residual neural network [3] is one of the most groundbreaking works in the computer vision and deep learning community. The ResNet makes it possible to train up to hundreds or even thousands of layers and still achieves compelling performance. It achieve this by introducing a identity shortcut connection that jumps over one or more layers. This indicates that the deeper model should not produce a training error higher than its shallower counterparts. The architecture of the ResNet is:

<img src="./figs/ResNet.png" width="700">

In this project, we use the ResNet18 model. The implementation of CNN-ResNet18 model is from [torchvision](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py) and we implemented the DCF-ResNet18 by replacing the 3x3 convolutional layers by the 3x3 DCF convolutional layers.

## 4  Experiments and Results
In this section, we conduct experiments with the models introduced before and report the results.  

*\*Since the model training is done with a GPU server where Jupyter Notebook is not available, we save the model after training and here we load the trained model and evaluate the classification accuracy. All the models are trained by ourselves, unless otherwise specified. To reproduce the training, one may use the following terminal commands with the scripts provided:*

```bash
python main.py --model <model_name> --num_bases <value of K> --save <path to save model>
```
*or one may download our training results from: [Google Drive](https://drive.google.com/drive/folders/1EYzNXtav3JUw4GE-nO2kpJUw3KgAJYRW?usp=sharing)*

&lt;model_name> can be one of the followings:
- cnn_vgg16bn: original VGG16 model with batch normalization
- dcf_vgg16bn: VGG16 model with DCF convolutional layers and batch normalization
- cnn_resnet18: original ResNet18 model
- dcf_resnet18: ResNet18 model with DCF convolutional layers

In [3]:
# prepare data for model evaluation
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])
test_transform = transforms.Compose([
    transforms.ToTensor(), 
    normalize
])
cnn_resnet18_transform = transforms.Compose([
    transforms.Resize([224, 224]), 
    transforms.ToTensor(), 
    normalize
])
test_loader = DataLoader(CIFAR10(data_dir, train=False, transform=test_transform), 
                         batch_size=128, num_workers=0)
cnn_resnet18_loader = DataLoader(CIFAR10(data_dir, train=False, transform=cnn_resnet18_transform),
                                 batch_size=128, num_workers=0)

In [49]:
# function to load models and evaluate accuracy
def load_checkpoint(model, checkpoint, state_key='state_dict'):
    if not checkpoint.is_file():
        raise FileNotFoundError('Specified checkpoint do not exist. Please train the model and save checkpoint first.')
    model.load_state_dict(torch.load(checkpoint)[state_key])
    return model.to(device)

def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_sz = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_sz))
        return res

class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def eval_acc(net, data_loader):
    num_params = sum(x.data.nelement() for x in net.parameters())
    net.eval()

    top1 = AverageMeter()
    top5 = AverageMeter()
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            acc1, acc5 = accuracy(outputs, labels, topk=(1, 5))

            top1.update(acc1[0], inputs.size(0))
            top5.update(acc5[0], inputs.size(0))

        return top1.avg.item(), top5.avg.item(), num_params

### 4.1 VGG16 Model
We train CNN-VGG16 and DCF-Net-VGG16 models in CIFAR10 dataset from scratch.

In [46]:
# VGG16 CNN Model
model = cnn_vgg16bn(num_classes=10)
model.classifier[0] = nn.Linear(512, 4096)
net = load_checkpoint(model,
                      results_dir / 'cnn_vgg16bn.pt', 
                      state_key='model_state_dict')
top1, top5, num_params = eval_acc(net, test_loader)
print('VGG16 CNN Model')
print(f'Accuracy: top1={top1:.2f}%, top5={top5:.2f}%\n# Parameters: {num_params:.2e}')

VGG16 CNN Model
Accuracy: top1=89.15%, top5=99.63%
# Parameters: 3.36e+07


In [63]:
# VGG16 DCF Model with K=3
net = load_checkpoint(dcf_vgg16bn(num_bases=3, num_classes=10),
                      results_dir / 'dcf_vgg16bn_K3.pt', 
                      state_key='model_state_dict')
top1, top5, num_params = eval_acc(net, test_loader)
print('VGG16 DCF Model with K=3')
print(f'Accuracy: top1={top1:.2f}%, top5={top5:.2f}%\n# Parameters: {num_params:.2e}')

VGG16 DCF Model with K=3
Accuracy: top1=91.02%, top5=99.72%
# Parameters: 2.38e+07


In [58]:
# VGG16 DCF Model with K=5
net = load_checkpoint(dcf_vgg16bn(num_bases=5, num_classes=10),
                      results_dir / 'dcf_vgg16bn_K5.pt', 
                      state_key='state_dict')
top1, top5, num_params = eval_acc(net, test_loader)
print('VGG16 DCF Model with K=5')
print(f'Accuracy: top1={top1:.2f}%, top5={top5:.2f}%\n# Parameters: {num_params:.2e}')

VGG16 DCF Model with K=5
Accuracy: top1=88.07%, top5=99.50%
# Parameters: 2.71e+07


### 4.2 ResNet18 Model

In [32]:
# ResNet18 CNN Model
net = load_checkpoint(cnn_resnet18(num_classes=10), 
                      results_dir / 'cnn_resnet18.pt')
top1, top5, num_params = eval_acc(net, cnn_resnet18_loader)
print('ResNet18 CNN Model')
print(f'Accuracy: top1={top1:.2f}%, top5={top5:.2f}%\n# Parameters: {num_params:.2e}')

ResNet18 CNN Model
Accuracy: top1=88.64%, top5=99.46%
# Parameters: 1.12e+07


In [33]:
# ResNet18 DCF Model with K=3
net = load_checkpoint(dcf_resnet18(num_bases=3, num_classes=10), 
                      results_dir / 'dcf_resnet18_K3.pt')
top1, top5, num_params = eval_acc(net, test_loader)
print('ResNet18 DCF Model with K=3')
print(f'Accuracy: top1={top1:.2f}%, top5={top5:.2f}%\n# Parameters: {num_params:.2e}')

ResNet18 DCF Model with K=3
Accuracy: top1=84.33%, top5=99.26%
# Parameters: 3.85e+06


In [34]:
# ResNet18 DCF Model with K=5
net = load_checkpoint(dcf_resnet18(num_bases=5, num_classes=10), 
                      results_dir / 'dcf_resnet18_K5.pt')
top1, top5, num_params = eval_acc(net, test_loader)
print('ResNet18 DCF Model with K=5')
print(f'Accuracy: top1={top1:.2f}%, top5={top5:.2f}%\n# Parameters: {num_params:.2e}')

ResNet18 DCF Model with K=5
Accuracy: top1=86.09%, top5=99.35%
# Parameters: 6.29e+06


### 4.3 Summary of Results


<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-ydu8{font-size:16px;font-family:"Times New Roman", Times, serif !important;;text-align:center}
.tg .tg-plqh{font-size:16px;font-family:"Times New Roman", Times, serif !important;;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-ydu8" colspan="4">CIFAR-10, kernel size=3x3</th>
  </tr>
  <tr>
    <td class="tg-ydu8"></td>
    <td class="tg-ydu8">Top1-Accuracy</td>
    <td class="tg-plqh">Top5-Accuracy</td>
    <td class="tg-plqh"># Model Parameters</td>
  </tr>
    
  <tr>
    <td class="tg-plqh">CNN-VGGNet-16_bn</td>
    <td class="tg-plqh">89.15%</td>
    <td class="tg-plqh">99.63%</td>
    <td class="tg-plqh">3.36e+07</td>
  </tr>
  <tr>
    <td class="tg-plqh">DCF-VGGNet (K=3)</td>
    <td class="tg-plqh"><strong>91.02%</strong></td>
    <td class="tg-plqh"><strong>99.72%</strong></td>
    <td class="tg-plqh"><strong>2.38e+07</strong></td>
  </tr>
  <tr>
    <td class="tg-plqh">DCF-VGGNet (K=5)</td>
    <td class="tg-plqh">88.07%</td>
    <td class="tg-plqh">99.50%</td>
    <td class="tg-plqh">2.71e+07</td>
  </tr>
  <tr>
    <td class="tg-plqh"></td>
    <td class="tg-plqh"></td>
    <td class="tg-plqh"></td>
    <td class="tg-plqh"></td>
  </tr> 
  <tr>
    <td class="tg-plqh">CNN-ResNet18</td>
    <td class="tg-plqh"><strong>88.64%</strong></td>
    <td class="tg-plqh"><strong>99.46%</strong></td>
    <td class="tg-plqh">1.12e+07</td>
  </tr>
  <tr>
    <td class="tg-plqh">DCF-ResNet18 (K=3)</td>
    <td class="tg-plqh">84.33%</td>
    <td class="tg-plqh">99.26%</td>
    <td class="tg-plqh"><strong>3.85e+06</strong></td>
  </tr>
  <tr>
    <td class="tg-plqh">DCF-ResNet18 (K=5)</td>
    <td class="tg-plqh">86.09%</td>
    <td class="tg-plqh">99.35%</td>
    <td class="tg-plqh">6.29e+06</td>
  </tr>
</table>

## 5 Conclusion and Discussion
### 5.1 Conclusion
In this project, we explored using the pre-fixed Fourier-Bessel (FB) bases to train a CNN where the coefficients of the bases are learned from data. We selected VGG16 and ResNet18 models as basic models and replaced the convolutional layers by DCF-FB layers. Experiments are carried out on CIFAR10 dataset. From the experiment results, we observe that the DCF-Nets are able to achieve comparable performance with the baseline CNN models with significantly less number of paramters, which is really impressive. Due to the significant decrease of number of parameters, the DCF-Nets can be trained much efficiently. Unfortunately, due to the time and GPU resources limit, we were not able to perform more comprehensive parameter tuning.

### 5.2 Discussion
Fourier-Bessel bases show capacity to capture the low-frequency components in the inputs and effectively ignore those high-frequency nuance details, which are often less stable and irrelevant for classification tasks. Other functional bases, such as standard Fourier bases or wavelet bases, may be introduced to improve model performance. Hopefully, the DCF layers can be incorporated in networks for learning the salient representations in an unsupervised manner. The stability and robustness analysis can be extended by testing the resilience to adversarial examples, which easily fool the state-of-the-art CNN architectures.

## References

[1] Alex Krizhevsky, Ilya Sutskever and Geoffrey E. Hinton. ImageNet Classification with Deep Convolutional Neural Networks. In Advances in Neural Information Processing Systems, pp. 1097--1105, 2012.

[2] Karen Simonyan and Andrew Zisserman. Very Deep Convolutional Networks for Large-Scale Image Recognition. In International Conference on Learning Representations, 2015.

[3] Kaiming He, Xiangyu Zhang, Shaoqing Ren and Jian Sun. Deep Residual Learning for Image Recognition. In IEEE Conference on Computer Vision and Pattern Recognition, pp. 770--778, 2016.

[4] Gao Huang, Zhuang Liu, Laurens van der Maaten and Kilian Q. Weinberger. Densely Connected Convolutional Networks. In IEEE Conference on Computer Vision and Pattern Recognition, pp. 2261--2269, 2017.

[5] Joan Bruna and Stéphane Mallat. Invariant Scattering Convolution Networks. IEEE Transactions on Pattern Analysis and Machine Intelligence, vol. 35, no. 8, pp. 1872--1886, 2012.

[6] Qiang Qiu, Xiuyuan Cheng, Robert Calderbank and Guillermo Sapiro. DCFNet: Deep Neural Network with Decomposed Convolutional Filters. In International Conference on Machine Learning, pp. 4198--4207, 2018.